A simplified workthrough of Mask RCNN training. Currently contains no evaluation.

In [1]:
import os
import sys
import itertools
from statistics import mean
from time import time
from tqdm.notebook import tqdm
import numpy as np
sys.path.append('..')

Set XLA to fusible. This will fuse smaller operations, but leave larger operations as is. So far, this seems to give the biggest performance gains.

In [2]:
os.environ['TF_XLA_FLAGS'] = "--tf_xla_auto_jit=fusible"

Disable all TF 2 behavior. Set TF to use GPU 0.

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.disable_v2_behavior()
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

import horovod.tensorflow as hvd
hvd.init()

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[hvd.rank()], 'GPU')
devices = tf.config.list_logical_devices('GPU')

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from mask_rcnn.hyperparameters import dataset_params
from mask_rcnn.hyperparameters import mask_rcnn_params
from mask_rcnn import dataset_utils
import load_weights, model
from evaluation import compute_coco_eval_metric_nonestimator, process_prediction_for_eval

Train data can be found at s3://[redacted]/data/coco/tfr_anchor/

In [5]:
train_file_pattern = '/home/ubuntu/data/train*'
batch_size = 6
data_params = dataset_params.get_data_params()
params = mask_rcnn_params.default_config().values()
data_params['batch_size'] = batch_size
params['finetune_bn'] = False
params['train_batch_size'] = batch_size
params['l2_weight_decay'] = 1e-4
params['init_learning_rate'] = 1e-4 * batch_size
params['warmup_learning_rate'] = 1e-3 * batch_size
params['warmup_steps'] = 500
params['learning_rate_steps'] = [30000,40000]
params['learning_rate_levels'] = [1e-4 * batch_size, 1e-5 * batch_size]
params['momentum'] = 0.9
params['use_batched_nms'] = True

In [6]:
data_params_eval = dataset_params.get_data_params()
data_params_eval['batch_size'] = 8

Fast data loader is a modified data loader that handles precomputed box matches.

In [7]:
train_input_fn = dataset_utils.FastDataLoader(train_file_pattern, data_params)
train_tdf = train_input_fn(data_params)
tdf_iter = train_tdf.make_initializable_iterator()
features, labels = tdf_iter.get_next()

[MaskRCNN] INFO    : Using Dataset Sharding with Horovod
Instructions for updating:
This is a deprecated API that should only be used in TF 1 graph mode and legacy TF 2 graph mode available through `tf.compat.v1`. In all other situations -- namely, eager mode and inside `tf.function` -- you can consume dataset elements using `for elem in dataset: ...` or by explicitly creating iterator via `iterator = iter(dataset)` and fetching its elements via `values = next(iterator)`. Furthermore, this API is not available in TF 2. During the transition from TF 1 to TF 2 you can use `tf.compat.v1.data.make_initializable_iterator(dataset)` to create a TF 1 graph mode style iterator for a dataset created through TF 2 APIs. Note that this should be a transient state of your code base as there are in general no guarantees about the interoperability of TF 1 and TF 2 code.


In [8]:
val_file_pattern = "/home/ubuntu/data/val*"
val_input_fn = dataset_utils.FastDataLoader(val_file_pattern, data_params_eval)
val_tdf = val_input_fn(data_params_eval)
val_iter = val_tdf.make_initializable_iterator()
features_val, labels_val = val_iter.get_next()

[MaskRCNN] INFO    : Using Dataset Sharding with Horovod


In [9]:
train_op, total_loss = model.model(features, params, labels)
model_output = model.model(features_val, params, labels_val, False)

Load pretrained resnet weights. Make sure to first run the ./download_and_process_pretrained_weights.sh script.

In [10]:
var_list = load_weights.build_assigment_map('resnet50/')
checkpoint_file = tf.train.latest_checkpoint('../resnet/resnet-nhwc-2018-02-07/')
_init_op, _init_feed_dict = load_weights.assign_from_checkpoint(checkpoint_file, var_list)

Training loop. To run multi-gpu with horovod, use the run.sh script.

In [ ]:
var_initializer = tf.global_variables_initializer()
loss_history = []
steps = 118000//(batch_size * hvd.size())
#steps = 10
val_json_file="/home/ubuntu/nv_tfrecords/annotations/instances_val2017.json"
with tf.Session() as sess:
    sess.run(_init_op, _init_feed_dict)
    sess.run(tdf_iter.initializer)
    sess.run(val_iter.initializer)
    sess.run(var_initializer)
    for epoch in range(1):
        if hvd.rank()==0:
            progressbar = tqdm(range(steps))
            loss_history = []
        else:
            progressbar = range(steps)
        for i in progressbar:
            op, loss = sess.run((train_op, total_loss))
            if hvd.rank()==0:
                loss_history.append(loss)
                progressbar.set_description("Loss: {0:.4f}".format(np.array(loss_history[-50:]).mean()))
        
        eval_batch_size = 8
        eval_steps = 5000//(eval_batch_size * hvd.size())
        #eval_steps = 625
        progressbar_eval = tqdm(range(eval_steps))
        
        predictions = dict()
        for i in progressbar_eval:
            out= sess.run((model_output))
            out = process_prediction_for_eval(out)

            for k, v in out.items():
                if k not in predictions:
                    predictions[k] = [v]
                else:
                    predictions[k].append(v)
         
        #print(predictions.keys())
        compute_coco_eval_metric_nonestimator(predictions, annotation_json_file=val_json_file)


[GPU 00] Restoring pretrained weights (265 Tensors) from: ../resnet/resnet-nhwc-2018-02-07/model.ckpt-112603


In [7]:
import pickle
import dllogger
val_json_file="/home/ubuntu/nv_tfrecords/annotations/instances_val2017.json"
f = np.load("/home/ubuntu/DeepLearningExamples_Anurag/TensorFlow2/Segmentation/MaskRCNN/scripts/test.npy", allow_pickle=True).item()
#pred=np.load("pred.npy", allow_pickle=True).item()
# file = open("/home/ubuntu/DeepLearningExamples_Anurag/TensorFlow2/Segmentation/MaskRCNN/scripts/file.pkl", 'rb')
# data = pickle.load(file)
# # close the file
# file.close()

#print(data.dtype)
compute_coco_eval_metric_1(pred, annotation_json_file=val_json_file, num_batches=625,
                                  eval_batch_size = 8)



loading annotations into memory...
Done (t=0.62s)
creating index...
index created!
[MaskRCNN] INFO    : Running inference on batch 001/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 002/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 003/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 004/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 005/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 006/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 007/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 008/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 009/625... - Step Time: 0.000

[MaskRCNN] INFO    : Running inference on batch 079/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 080/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 081/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 082/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 083/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 084/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 085/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 086/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 087/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 088/625

[MaskRCNN] INFO    : Running inference on batch 158/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 159/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 160/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 161/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 162/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 163/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 164/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 165/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 166/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 167/625

[MaskRCNN] INFO    : Running inference on batch 237/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 238/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 239/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 240/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 241/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 242/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 243/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 244/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 245/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 246/625

[MaskRCNN] INFO    : Running inference on batch 316/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 317/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 318/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 319/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 320/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 321/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 322/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 323/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 324/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 325/625

...

[MaskRCNN] INFO    : Running inference on batch 474/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 475/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 476/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 477/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 478/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 479/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 480/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 481/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 482/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 483/625

[MaskRCNN] INFO    : Running inference on batch 553/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 554/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 555/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 556/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 557/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 558/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 559/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 560/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 561/625... - Step Time: 0.0000s - Throughput: 0.0 imgs/s
[MaskRCNN] INFO    : Running inference on batch 562/625

[MaskRCNN] INFO    : Loading and preparing results...
[MaskRCNN] INFO    : 0/375000
[MaskRCNN] INFO    : 1000/375000
[MaskRCNN] INFO    : 2000/375000
[MaskRCNN] INFO    : 3000/375000
[MaskRCNN] INFO    : 4000/375000
[MaskRCNN] INFO    : 5000/375000
[MaskRCNN] INFO    : 6000/375000
[MaskRCNN] INFO    : 7000/375000
[MaskRCNN] INFO    : 8000/375000
[MaskRCNN] INFO    : 9000/375000
[MaskRCNN] INFO    : 10000/375000
[MaskRCNN] INFO    : 11000/375000
[MaskRCNN] INFO    : 12000/375000
[MaskRCNN] INFO    : 13000/375000
[MaskRCNN] INFO    : 14000/375000
[MaskRCNN] INFO    : 15000/375000
[MaskRCNN] INFO    : 16000/375000
[MaskRCNN] INFO    : 17000/375000
[MaskRCNN] INFO    : 18000/375000
[MaskRCNN] INFO    : 19000/375000
[MaskRCNN] INFO    : 20000/375000
[MaskRCNN] INFO    : 21000/375000
[MaskRCNN] INFO    : 22000/375000
[MaskRCNN] INFO    : 23000/375000
[MaskRCNN] INFO    : 24000/375000
[MaskRCNN] INFO    : 25000/375000
[MaskRCNN] INFO    : 26000/375000
[MaskRCNN] INFO    : 27000/375000
[MaskRC

[MaskRCNN] INFO    : 236000/375000
[MaskRCNN] INFO    : 237000/375000
[MaskRCNN] INFO    : 238000/375000
[MaskRCNN] INFO    : 239000/375000
[MaskRCNN] INFO    : 240000/375000
[MaskRCNN] INFO    : 241000/375000
[MaskRCNN] INFO    : 242000/375000
[MaskRCNN] INFO    : 243000/375000
[MaskRCNN] INFO    : 244000/375000
[MaskRCNN] INFO    : 245000/375000
[MaskRCNN] INFO    : 246000/375000
[MaskRCNN] INFO    : 247000/375000
[MaskRCNN] INFO    : 248000/375000
[MaskRCNN] INFO    : 249000/375000
[MaskRCNN] INFO    : 250000/375000
[MaskRCNN] INFO    : 251000/375000
[MaskRCNN] INFO    : 252000/375000
[MaskRCNN] INFO    : 253000/375000
[MaskRCNN] INFO    : 254000/375000
[MaskRCNN] INFO    : 255000/375000
[MaskRCNN] INFO    : 256000/375000
[MaskRCNN] INFO    : 257000/375000
[MaskRCNN] INFO    : 258000/375000
[MaskRCNN] INFO    : 259000/375000
[MaskRCNN] INFO    : 260000/375000
[MaskRCNN] INFO    : 261000/375000
[MaskRCNN] INFO    : 262000/375000
[MaskRCNN] INFO    : 263000/375000
[MaskRCNN] INFO    :

DLLoggerNotInitialized: DLLogger not initialized. Initialize DLLogger with init(backends) function

In [41]:
np.save("predictions.npy", predictions)

In [44]:
out['detection_boxes'].shape

(6, 100, 4)